全ての指標を関数化させる

In [21]:
import pandas as pd
from numpy import nan

def slide(present):
    past = present.shift(-1)
    growth_rate  = (present - past)/past *100
    return growth_rate.dropna()

In [22]:
def w_ave(x):
    sum_wa = 0
    count=0
    
    #欠損値を削除
    x_dropna = x.dropna()
    
    #シリーズ型のためvalueのみ取り出している
    for quarter in x_dropna.values:
        count += 1
        weight = quarter * count
        sum_wa += weight 
        
    if sum_wa == 0 :
        return nan
    else:
        return sum_wa / ((1/2)*count*(count+1))

In [23]:
#第2章のデータのクレンジングで使用
def w_ave(x):
    sum_wa = 0
    count=0
    
    #欠損値を削除
    x_dropna = x.dropna()
    
    #シリーズ型のためvalueのみ取り出している
    for quarter in x_dropna.values:
        count += 1
        weight = quarter * count
        sum_wa += weight 
        
    if sum_wa == 0 :
        return nan
    else:
        result = sum_wa / ((1/2)*count*(count+1))
        if result == float('inf') :
            return nan
        else :
            return result

In [24]:
def growth_potential(df_q):
    #売上成長
    gp1_slide = slide(df_q["net_sales"])
    gp1_nsgr =w_ave(gp1_slide)
    #営業利益成長率
    gp2_slide = slide(df_q["operating_income"])
    gp2_oigr =w_ave(gp2_slide)
    return  gp1_nsgr, gp2_oigr

In [25]:
def profitability(df_q):
    #ROE
    pr1_roe = w_ave(df_q["roe"])  
    #営業利益率
    pr2_om = w_ave(df_q["operating_margin"]) 
    return pr1_roe, pr2_om

In [26]:
def labor_productivity(df_q):
    #1人当たり売上高
    lp1_nspe = w_ave(df_q["net_sales_per_employee"])
    #1人当たり営業CF
    lp2_oipe = w_ave(df_q["operating_income_per_employee"])
    return lp1_nspe, lp2_oipe

In [27]:
def efficiency(df_q):
    #総資産回転率
    e1_tat = w_ave(df_q["total_asset_turnover"])
    #<2:CCC(キャッシュコンバージョンサイクル) = 売上債権回転期間 + 棚卸資産回転期間 – 支払債務回転期間)>
    e2_df = df_q[["accounts_receivable_turnover","inventory_turnover","trade_payable_turnover"]]
    e2_ccc = w_ave(e2_df["trade_payable_turnover"])+w_ave(e2_df["inventory_turnover"])-w_ave(e2_df["accounts_receivable_turnover"])
    return e1_tat,e2_ccc

In [28]:
def safety(df_q):
    #自己資本比率 equity_ratio
    s1_er = w_ave(df_q["equity_ratio"])
    #ネットD/Eレシオ = 純有利子負債 net_debt/ 自己資本 equity
    s2_df = df_q[["net_debt","equity"]]
    s2_nder = w_ave(s2_df["net_debt"]/s2_df["equity"])
    return  s1_er, s2_nder

In [29]:
def shareholder_returnability(df_q):
    #<1:配当性向 Dividend payout ratio (%）＝ 1株あたり配当金 Dividends per share ÷1株あたり純利益（EPS）×100>
    sr1_df = df_q[["dividend","eps_actual"]]
    sr1_dpr = w_ave(sr1_df["dividend"] /sr1_df["eps_actual"] * 100)
    #<2:株主資本配当率 DOE(株主資本配当率）＝ 年間総配当額 ÷ 自己資本>
    sr2_doe = w_ave(df_q["doe"])
    return sr1_dpr,sr2_doe

In [30]:
def dc_merge(daily,current):
    if current[0] is None:
        return daily
    else : 
        dcm = pd.concat([daily,current],axis = 0)
        return dcm

def capital_gain(df_day,df_c):
    df_cg = df_day[["market_capital","num_of_shares"]]
    cg_d = df_cg["market_capital"]/df_cg["num_of_shares"]
    cg_c = df_c["stockprice"]
    cg_merge = dc_merge(cg_d,cg_c)
    cg_before = cg_merge
    cg_after = cg_merge.shift(-1)
    #slide関数では100倍するため注意
    cg_ab = (cg_after - cg_before)/cg_before
    cg = w_ave(cg_ab)
    return cg

In [16]:
#取得したCSVデータを読み込む際に、もとの証券コードのリストを頼りにしている
ticker_list = input("加工したい元データを記入してください")

with open(f"ticker_list/{ticker_list}.txt" ,mode = "r") as f:
    file = f.read()
ticker_list_three = file.split("\n")

if ticker_list == "ticker" :
    with open("ticker_list/ticker_count.txt" , mode = "r") as f:
        count_int = int(f.read())
    count = count_int
    ticker_data = ",".join(ticker_list_three[:count])
    ticker_data = ticker_data.split(",")

else :
    ticker_data = ",".join(ticker_list_three)
    ticker_data = ticker_data.split(",")
    
print("取得が完了しました")

加工したい元データを記入してくださいticker
取得が完了しました


In [ ]:
ticker_data

In [ ]:
processed_df = []

for i in range(len(ticker_data)):
    ticker = ticker_data[i]
    if not ticker.isdecimal():
        break
    df_q = pd.read_csv(f"df_q/{ticker_data[i]}.csv")
    df_day = pd.read_csv(f"df_day/{ticker_data[i]}.csv")
    df_c = pd.read_csv(f"df_c/{ticker_data[i]}.csv")

    
    gp1_nsgr, gp2_oigr = growth_potential(df_q)
    pr1_roe, pr2_om = profitability(df_q)
    lp1_nspe, lp2_oipe= labor_productivity(df_q)
    e1_tat,e2_ccc = efficiency(df_q)
    s1_er, s2_nder = safety(df_q)
    sr1_dpr,sr2_doe = shareholder_returnability(df_q)
    cg = capital_gain(df_day,df_c)
    

    company = df_q.loc[ :,"company_name"]
    df_q.index.name = company[0]
    df_list =pd.DataFrame({"company":[df_q.index.name],
                        "code":[ticker],
                        "sales_growth_rate":[gp1_nsgr],
                        "operating_income_growth_rate":[gp2_oigr],
                        "roe":[pr1_roe],
                        "operating_margin":[pr2_om],
                        "net_sales_per_employee":[lp1_nspe],
                        "operating_income_per_employee":[lp2_oipe],
                         "total_assets_turnover":[e1_tat],
                        "ccc":[e2_ccc],
                       "equity_ratio":[s1_er],
                        "net_d/e_ratio":[s2_nder],
                        "dividend_payout_ratio":[sr1_dpr],
                         "doe":[sr2_doe],
                        "capital_gain":[cg]
                        })
    
    processed_df.append(df_list)
    
processed_df

In [18]:
#データを保存する
processed_data = pd.concat([processed_df[i] for i in range(len(ticker_data)-1)])
processed_data.to_csv("ProcessedData.csv",index = False,encoding = "utf_8_sig")

In [ ]:
#データを読み込む
processed_data = pd.read_csv("ProcessedData.csv")
processed_data

In [ ]:
#欠損値があるか調べる
processed_data.isnull().sum()

In [ ]:
#nanのある企業を特定 (補足 企業を見たい方向け)
data_nan = processed_data[processed_data.isnull().any(1)]
data_nan

In [ ]:
#欠損値を埋める
processed_data = processed_data.fillna(processed_data.median())
processed_data.isnull().sum()

In [ ]:
# infがあるか確認
for i in  processed_data.columns:
    print(i, len(processed_data[processed_data[f"{i}"] == float("inf")] ))

In [ ]:
#infのある行を特定  (補足 企業を見たい方向け)
data_inf =pd.DataFrame()
for i in  processed_data.columns:
    pd_inf = processed_data[processed_data[f"{i}"] == float("inf")]
    data_inf = pd.concat([data_inf,pd_inf],axis = 0)
    
data_inf

In [ ]:
#各指標を確認すると、下記の指標にinfがあることがわかる

In [ ]:
# operating_income_growth_rate 
gp2_slide = slide(df_q["operating_income"])
gp2_oigr =w_ave(gp2_slide)

# net_d/e_ratio 
s2_df = df_q[["net_debt","equity"]]
s2_nder = w_ave(s2_df["net_debt"]/s2_df["equity"])

# dividend_payout_ratio 
sr1_df = df_q[["dividend","eps_actual"]]
sr1_dpr = w_ave(sr1_df["dividend"] /sr1_df["eps_actual"] * 100)

# capital_gain 
cg_ab = (cg_after - cg_before)/cg_before
cg = w_ave(cg_ab)

In [1]:
#加重平均関数を修正する
def w_ave(x):
    sum_wa = 0
    count=0
    
    #欠損値を削除
    x_dropna = x.dropna()
    
    #シリーズ型のためvalueのみ取り出している
    for quarter in x_dropna.values:
        count += 1
        weight = quarter * count
        sum_wa += weight 
        
    if sum_wa == 0 :
        return nan
    else:
        result = sum_wa / ((1/2)*count*(count+1))
        if result == float('inf') :
            return nan
        else :
            return result

In [ ]:
#加重平均関数を修正した上で、データの加工をしましょう！
# nanに関しては 保存前に一括削除
# infに関しては w_ave時点で削除

In [ ]:
""""
修正した加重平均関数を実行し、
全12指標とキャピタルゲイン関数を実行し、
銘柄コードを指定するticker_dataを出力したら、
こちらのセルに移動し、下以降のセルを実行。
"""
processed_df = []

for i in range(len(ticker_data)):
    ticker = ticker_data[i]
    if not ticker.isdecimal():
        break
    df_q = pd.read_csv(f"df_q/{ticker_data[i]}.csv")
    df_day = pd.read_csv(f"df_day/{ticker_data[i]}.csv")
    df_c = pd.read_csv(f"df_c/{ticker_data[i]}.csv")

    
    gp1_nsgr, gp2_oigr = growth_potential(df_q)
    pr1_roe, pr2_om = profitability(df_q)
    lp1_nspe, lp2_oipe= labor_productivity(df_q)
    e1_tat,e2_ccc = efficiency(df_q)
    s1_er, s2_nder = safety(df_q)
    sr1_dpr,sr2_doe = shareholder_returnability(df_q)
    cg = capital_gain(df_day,df_c)
    

    company = df_q.loc[ :,"company_name"]
    df_q.index.name = company[0]
    df_list =pd.DataFrame({"company":[df_q.index.name],
                        "code":[ticker],
                        "sales_growth_rate":[gp1_nsgr],
                        "operating_income_growth_rate":[gp2_oigr],
                        "roe":[pr1_roe],
                        "operating_margin":[pr2_om],
                        "net_sales_per_employee":[lp1_nspe],
                        "operating_income_per_employee":[lp2_oipe],
                         "total_assets_turnover":[e1_tat],
                        "ccc":[e2_ccc],
                       "equity_ratio":[s1_er],
                        "net_d/e_ratio":[s2_nder],
                        "dividend_payout_ratio":[sr1_dpr],
                         "doe":[sr2_doe],
                        "capital_gain":[cg]
                        })
    
    processed_df.append(df_list)
    
processed_df

In [34]:
processed_data = pd.concat([processed_df[i] for i in range(len(ticker_data)-1)])
processed_data = processed_data.fillna(processed_data.median())
#欠損値を埋めて保存
processed_data.to_csv("ProcessedData.csv",index = False,encoding = "utf_8_sig")

In [ ]:
#再度読み込む
processed_data = pd.read_csv("ProcessedData.csv")
processed_data

In [ ]:
#再度確認
processed_data.isnull().sum()

In [ ]:
#再度確認
for i in  processed_data.columns:
    print(i, len(processed_data[processed_data[f"{i}"] == float("inf")] ))